# Vox Amelior (Nemo Server) - Google Colab Setup

This notebook deploys the Vox Amelior platform on Google Colab.

**Prerequisites:**
1.  Set `NGROK_AUTHTOKEN` in Colab Secrets (Key icon on the left).
2.  Set `SESSION_KEY` and `USERS_DB_KEY` (random strings) in Secrets.
3.  (Optional) Set `HF_TOKEN` for Gemma models.

## 1. Mount Drive & Clone Repository
We mount Google Drive first to enable caching, then clone/load the repo.

In [ ]:
# @title Mount Drive & Setup Repo
import os
import sys
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define paths
DRIVE_BASE = "/content/drive/MyDrive/Nemo_Server"
LIB_CACHE = os.path.join(DRIVE_BASE, "lib_cache")
REPO_PATH = os.path.join(DRIVE_BASE, "repo")

# 3. Create directories if needed
os.makedirs(LIB_CACHE, exist_ok=True)
os.makedirs(REPO_PATH, exist_ok=True)

# 4. Clone repo to Drive if not already there
if not os.path.exists(os.path.join(REPO_PATH, "services")):
    print("Cloning repository to Drive...")
    !git clone https://github.com/pruittcolon/Nemo_Server.git "$REPO_PATH"
else:
    print("Repository found on Drive.")

# 5. Change to repo directory
%cd $REPO_PATH
print(f"Working directory: {os.getcwd()}")

## 2. Install System Dependencies

In [ ]:
# @title Install System Dependencies
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 -qq
print("System dependencies installed.")

## 3. Install & Cache Python Dependencies
Dependencies are cached on Drive. First run takes a few minutes; subsequent runs are instant.

In [ ]:
# @title Install & Cache Dependencies
import os
import sys

# Add cache to Python path
LIB_CACHE = "/content/drive/MyDrive/Nemo_Server/lib_cache"
if LIB_CACHE not in sys.path:
    sys.path.insert(0, LIB_CACHE)

# Check if cache is populated
cache_files = os.listdir(LIB_CACHE) if os.path.exists(LIB_CACHE) else []

if len(cache_files) < 5:  # More robust check than empty
    print("Installing dependencies to Drive cache (first run)...")
    
    # Install core packages to cache (skip torch/numpy - use Colab's)
    !pip install -t "/content/drive/MyDrive/Nemo_Server/lib_cache" \
        fastapi uvicorn pydantic httpx pyngrok nest_asyncio \
        python-multipart python-jose bcrypt cryptography \
        soundfile librosa ffmpeg-python redis \
        --quiet
    
    print("Dependencies cached to Drive.")
else:
    print(f"Cache found ({len(cache_files)} items). Skipping install.")

## 4. Configure Environment

In [ ]:
# @title Load Secrets & Configure Services
import os
from google.colab import userdata

# Load secrets from Colab
os.environ["NGROK_AUTHTOKEN"] = userdata.get('NGROK_AUTHTOKEN')
os.environ["SESSION_KEY"] = userdata.get('SESSION_KEY')
os.environ["USERS_DB_KEY"] = userdata.get('USERS_DB_KEY')

# Service URLs (localhost mode - no Docker)
os.environ["GEMMA_URL"] = "http://localhost:8001"
os.environ["TRANSCRIPTION_URL"] = "http://localhost:8003"
os.environ["RAG_URL"] = "http://localhost:8004"
os.environ["EMOTION_URL"] = "http://localhost:8005"
os.environ["INSIGHTS_URL"] = "http://localhost:8010"

# Optional HF token
try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
except:
    pass

print("Environment configured.")

## 5. Start API Gateway

In [ ]:
# @title Start API Gateway
import threading
import time
import sys
import os

# Ensure paths are set
sys.path.insert(0, os.getcwd())
sys.path.insert(0, os.path.join(os.getcwd(), "services/api-gateway"))

# Patch asyncio for Jupyter
import nest_asyncio
nest_asyncio.apply()

def run_gateway():
    import uvicorn
    uvicorn.run(
        "services.api-gateway.src.main:app",
        host="0.0.0.0",
        port=8000,
        log_level="warning"
    )

# Start in background thread
thread = threading.Thread(target=run_gateway, daemon=True)
thread.start()

time.sleep(5)
print("API Gateway started on port 8000.")

## 6. Create Public URL

In [ ]:
# @title Start Ngrok Tunnel
import os
from pyngrok import ngrok

ngrok.set_auth_token(os.environ["NGROK_AUTHTOKEN"])

public_url = ngrok.connect(8000).public_url
print(f"")
print(f"=" * 50)
print(f"DEPLOYMENT SUCCESSFUL")
print(f"=" * 50)
print(f"")
print(f"Access your app at: {public_url}/ui/login.html")
print(f"")